In [1]:
import pandas as pd
import numpy as np
import category_encoders as ce
import joblib as jb
from sklearn.model_selection import GridSearchCV

In [2]:
dataset = pd.read_csv('male_players (legacy).csv', low_memory=False)
dataset

,player_id,player_url,fifa_version,fifa_update,fifa_update_date,short_name,long_name,player_positions,overall,potential,...,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,player_face_url
0,158023,/player/158023/lionel-messi/150002,15,2,2014-09-18,L. Messi,Lionel Andrés Messi Cuccittini,CF,93,95,...,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3,https://cdn.sofifa.net/players/158/023/15_120.png
1,20801,/player/20801/c-ronaldo-dos-santos-aveiro/150002,15,2,2014-09-18,Cristiano Ronaldo,Cristiano Ronaldo dos Santos Aveiro,"LW, LM",92,92,...,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3,https://cdn.sofifa.net/players/020/801/15_120.png
2,9014,/player/9014/arjen-robben/150002,15,2,2014-09-18,A. Robben,Arjen Robben,"RM, LM, RW",90,90,...,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3,https://cdn.sofifa.net/players/009/014/15_120.png
3,41236,/player/41236/zlatan-ibrahimovic/150002,15,2,2014-09-18,Z. Ibrahimović,Zlatan Ibrahimović,ST,90,90,...,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3,https://cdn.sofifa.net/players/041/236/15_120.png
4,167495,/player/167495/manuel-neuer/150002,15,2,2014-09-18,M. Neuer,Manuel Peter Neuer,GK,90,90,...,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3,https://cdn.sofifa.net/players/167/495/15_120.png
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,269011,/player/269011/ziye-zhao/230002,23,2,2022-09-26,Zhao Ziye,赵梓业,RW,46,61,...,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2,https://cdn.sofifa.net/players/269/011/23_120.png
161579,269019,/player/269019/junjie-wu/230002,23,2,2022-09-26,Wu Junjie,吴俊杰,LB,46,58,...,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2,https://cdn.sofifa.net/players/269/019/23_120.png
161580,271093,/player/271093/kailin-barlow/230002,23,2,2022-09-26,K. Barlow,Kailin Barlow,CDM,46,58,...,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2,https://cdn.sofifa.net/players/271/093/23_120.png
161581,271555,/player/271555/gideon-tetteh/230002,23,2,2022-09-26,G. Tetteh,Gideon Tetteh,"RW, LW",46,70,...,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2,https://cdn.sofifa.net/players/271/555/23_120.png


In [3]:
dataset.describe()

,player_id,fifa_version,fifa_update,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,...,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed
count,161583.000000,161583.000000,161583.0,161583.000000,161583.000000,1.595300e+05,159822.000000,161583.000000,161583.000000,161583.000000,...,128722.000000,161583.000000,161583.000000,161583.000000,161583.00000,161583.000000,161583.000000,161583.000000,161583.000000,17969.000000
mean,214484.722353,19.125514,2.0,65.699071,70.744008,2.326770e+06,10855.409768,25.123181,181.240205,75.235031,...,57.816892,45.757957,47.669996,45.698588,16.52961,16.274918,16.140374,16.288861,16.636973,39.149090
std,34928.608856,2.559318,0.0,7.040855,6.259121,6.005746e+06,21941.656285,4.670207,6.750148,7.000456,...,12.329739,20.453699,21.336404,20.935273,17.67047,16.834294,16.476466,16.998697,17.980143,10.503788
min,2.000000,15.000000,2.0,40.000000,40.000000,1.000000e+03,500.000000,16.000000,154.000000,49.000000,...,3.000000,1.000000,2.000000,3.000000,1.00000,1.000000,1.000000,1.000000,1.000000,12.000000
25%,199159.000000,17.000000,2.0,61.000000,66.000000,3.250000e+05,2000.000000,21.000000,176.000000,70.000000,...,50.000000,26.000000,27.000000,25.000000,8.00000,8.000000,8.000000,8.000000,8.000000,31.000000
50%,220621.000000,19.000000,2.0,66.000000,70.000000,7.250000e+05,4000.000000,25.000000,181.000000,75.000000,...,59.000000,50.000000,54.000000,52.000000,11.00000,11.000000,11.000000,11.000000,11.000000,41.000000
75%,236958.000000,21.000000,2.0,70.000000,75.000000,1.800000e+06,10000.000000,28.000000,186.000000,80.000000,...,66.000000,63.000000,66.000000,64.000000,14.00000,14.000000,14.000000,14.000000,14.000000,46.000000
max,271817.000000,23.000000,2.0,94.000000,95.000000,1.940000e+08,575000.000000,54.000000,208.000000,110.000000,...,96.000000,94.000000,94.000000,95.000000,91.00000,92.000000,95.000000,92.000000,94.000000,68.000000


In [4]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Columns: 110 entries, player_id to player_face_url
dtypes: float64(18), int64(45), object(47)
memory usage: 135.6+ MB


In [5]:
# The non-relevant columns to drop
# These columns would not be relevant to predict a player's overall rating


irrelevant_cols = ['player_id','player_url','fifa_version', 'fifa_update','fifa_update_date','dob', 'short_name','long_name', 'league_id','league_name', 'club_team_id', 'club_name', 'club_jersey_number', 'club_loaned_from', 'club_joined_date', 'club_contract_valid_until_year', 'nationality_id', 'nation_jersey_number', 'body_type', 'real_face', 'player_tags', 'player_traits', 'player_face_url']

In [6]:
dataset.drop(columns = irrelevant_cols, axis=True, inplace=True)
dataset

,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_position,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,CF,93,95,100500000.0,550000.0,27,169,67,1.0,CF,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,"LW, LM",92,92,79000000.0,375000.0,29,185,80,1.0,LW,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,1.0,SUB,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,ST,90,90,52500000.0,275000.0,32,195,95,1.0,ST,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,GK,90,90,63500000.0,300000.0,28,193,92,1.0,GK,...,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,RW,46,61,110000.0,700.0,18,180,73,1.0,RES,...,32+2,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2
161579,LB,46,58,110000.0,750.0,19,188,83,1.0,RES,...,41+2,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2
161580,CDM,46,58,110000.0,500.0,19,181,73,1.0,RES,...,45+2,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2
161581,"RW, LW",46,70,150000.0,500.0,17,175,68,1.0,RES,...,30+2,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2


In [7]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 87 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   player_positions             161583 non-null  object 
 1   overall                      161583 non-null  int64  
 2   potential                    161583 non-null  int64  
 3   value_eur                    159530 non-null  float64
 4   wage_eur                     159822 non-null  float64
 5   age                          161583 non-null  int64  
 6   height_cm                    161583 non-null  int64  
 7   weight_kg                    161583 non-null  int64  
 8   league_level                 158866 non-null  float64
 9   club_position                159810 non-null  object 
 10  nationality_name             161583 non-null  object 
 11  nation_team_id               9316 non-null    float64
 12  nation_position              9316 non-null    object 
 13 

In [8]:
#Dropping all columns that have more than 30% null values

L = []
L_less = []
for i in dataset.columns:
    if((dataset[i].isnull().sum()) < (0.3*(dataset.shape[0]))):
        L.append(i)
    else:
        L_less.append(i)

In [9]:
dataset = dataset[L]
dataset

,player_positions,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,club_position,...,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk
0,CF,93,95,100500000.0,550000.0,27,169,67,1.0,CF,...,62+3,62+3,62+3,62+3,54+3,45+3,45+3,45+3,54+3,15+3
1,"LW, LM",92,92,79000000.0,375000.0,29,185,80,1.0,LW,...,63+3,63+3,63+3,63+3,57+3,52+3,52+3,52+3,57+3,16+3
2,"RM, LM, RW",90,90,54500000.0,275000.0,30,180,80,1.0,SUB,...,64+3,64+3,64+3,64+3,55+3,46+3,46+3,46+3,55+3,14+3
3,ST,90,90,52500000.0,275000.0,32,195,95,1.0,ST,...,65+3,65+3,65+3,61+3,56+3,55+3,55+3,55+3,56+3,17+3
4,GK,90,90,63500000.0,300000.0,28,193,92,1.0,GK,...,40+3,40+3,40+3,36+3,36+3,38+3,38+3,38+3,36+3,87+3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,RW,46,61,110000.0,700.0,18,180,73,1.0,RES,...,32+2,32+2,32+2,37+2,36+2,31+2,31+2,31+2,36+2,14+2
161579,LB,46,58,110000.0,750.0,19,188,83,1.0,RES,...,41+2,41+2,41+2,43+2,44+2,48+2,48+2,48+2,44+2,15+2
161580,CDM,46,58,110000.0,500.0,19,181,73,1.0,RES,...,45+2,45+2,45+2,47+2,47+2,46+2,46+2,46+2,47+2,14+2
161581,"RW, LW",46,70,150000.0,500.0,17,175,68,1.0,RES,...,30+2,30+2,30+2,34+2,32+2,28+2,28+2,28+2,32+2,15+2


In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 83 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   player_positions             161583 non-null  object 
 1   overall                      161583 non-null  int64  
 2   potential                    161583 non-null  int64  
 3   value_eur                    159530 non-null  float64
 4   wage_eur                     159822 non-null  float64
 5   age                          161583 non-null  int64  
 6   height_cm                    161583 non-null  int64  
 7   weight_kg                    161583 non-null  int64  
 8   league_level                 158866 non-null  float64
 9   club_position                159810 non-null  object 
 10  nationality_name             161583 non-null  object 
 11  preferred_foot               161583 non-null  object 
 12  weak_foot                    161583 non-null  int64  
 13 

In [11]:
dataset =dataset.iloc[:, :56]

In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161583 entries, 0 to 161582
Data columns (total 56 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   player_positions             161583 non-null  object 
 1   overall                      161583 non-null  int64  
 2   potential                    161583 non-null  int64  
 3   value_eur                    159530 non-null  float64
 4   wage_eur                     159822 non-null  float64
 5   age                          161583 non-null  int64  
 6   height_cm                    161583 non-null  int64  
 7   weight_kg                    161583 non-null  int64  
 8   league_level                 158866 non-null  float64
 9   club_position                159810 non-null  object 
 10  nationality_name             161583 non-null  object 
 11  preferred_foot               161583 non-null  object 
 12  weak_foot                    161583 non-null  int64  
 13 

**separating the numeric and non-numeric data**

In [13]:
#Separate the quantitative and categorical variables
numeric_data = dataset.select_dtypes(include = np.number)
non_numeric = dataset.select_dtypes(include = ['object'])

In [14]:
# Multivariate imputation

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
imp = IterativeImputer(max_iter = 10, random_state = 0)
numeric_data = pd.DataFrame(np.round(imp.fit_transform(numeric_data)), columns = numeric_data.columns)

In [15]:
numeric_data

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,weak_foot,skill_moves,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
0,93.0,95.0,100500000.0,550000.0,27.0,169.0,67.0,1.0,3.0,4.0,...,76.0,91.0,25.0,21.0,20.0,6.0,11.0,15.0,14.0,8.0
1,92.0,92.0,79000000.0,375000.0,29.0,185.0,80.0,1.0,4.0,5.0,...,85.0,91.0,22.0,31.0,23.0,7.0,11.0,15.0,14.0,11.0
2,90.0,90.0,54500000.0,275000.0,30.0,180.0,80.0,1.0,2.0,4.0,...,80.0,90.0,29.0,26.0,26.0,10.0,8.0,11.0,5.0,15.0
3,90.0,90.0,52500000.0,275000.0,32.0,195.0,95.0,1.0,4.0,4.0,...,91.0,92.0,25.0,41.0,27.0,13.0,15.0,10.0,9.0,12.0
4,90.0,90.0,63500000.0,300000.0,28.0,193.0,92.0,1.0,4.0,1.0,...,37.0,58.0,25.0,25.0,25.0,87.0,85.0,92.0,90.0,86.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161578,46.0,61.0,110000.0,700.0,18.0,180.0,73.0,1.0,3.0,2.0,...,54.0,40.0,23.0,21.0,25.0,9.0,13.0,13.0,12.0,7.0
161579,46.0,58.0,110000.0,750.0,19.0,188.0,83.0,1.0,3.0,2.0,...,31.0,35.0,50.0,51.0,45.0,6.0,14.0,8.0,13.0,14.0
161580,46.0,58.0,110000.0,500.0,19.0,181.0,73.0,1.0,2.0,2.0,...,37.0,35.0,36.0,45.0,50.0,8.0,9.0,7.0,14.0,9.0
161581,46.0,70.0,150000.0,500.0,17.0,175.0,68.0,1.0,3.0,2.0,...,63.0,43.0,19.0,17.0,14.0,13.0,12.0,14.0,7.0,13.0


In [16]:
corr_matrix = numeric_data.corr()

#correlation between all the columns listed and overall rating
corr_matrix["overall"].sort_values(ascending=False)

overall                        1.000000
movement_reactions             0.845753
mentality_composure            0.699923
potential                      0.695362
wage_eur                       0.611957
value_eur                      0.567398
passing                        0.511909
attacking_short_passing        0.499757
mentality_vision               0.493282
international_reputation       0.485234
skill_long_passing             0.483808
power_shot_power               0.480297
physic                         0.465650
age                            0.458390
skill_ball_control             0.457011
dribbling                      0.418472
skill_curve                    0.415863
power_long_shots               0.409086
shooting                       0.399750
mentality_aggression           0.398310
attacking_crossing             0.396833
skill_fk_accuracy              0.387269
attacking_volleys              0.377048
skill_dribbling                0.369827
power_stamina                  0.360025


In [17]:
from sklearn.impute import SimpleImputer
# median for missing numeric values
# mean for missing object values
num_si = SimpleImputer(strategy='median')
obj_si = SimpleImputer(strategy='most_frequent')

In [18]:
numeric_scaled = num_si.fit_transform(numeric_data)
non_numeric_scaled = obj_si.fit_transform(non_numeric)


In [19]:
encoder = ce.BinaryEncoder(cols=non_numeric.columns)
col_encoded = encoder.fit_transform(non_numeric)
non_numeric.dropna(inplace=True)

In [20]:
dataset = pd.concat([numeric_data, non_numeric], axis=1).reset_index(drop=True)

In [21]:
corr_matrix = corr_matrix[corr_matrix.index]
corr_matrix

,overall,potential,value_eur,wage_eur,age,height_cm,weight_kg,league_level,weak_foot,skill_moves,...,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes
overall,1.000000,0.695362,0.567398,0.611957,0.458390,0.044415,0.140015,-0.224736,0.217422,0.330951,...,0.334488,0.699923,0.274173,0.250624,0.220398,-0.013603,-0.010862,-0.015521,-0.004731,-0.011783
potential,0.695362,1.000000,0.535377,0.511929,-0.201506,0.006744,-0.008111,-0.195368,0.167492,0.285126,...,0.225312,0.456100,0.152613,0.152078,0.134847,-0.044826,-0.044119,-0.048803,-0.042978,-0.044156
value_eur,0.567398,0.535377,1.000000,0.778832,0.066446,0.011040,0.035725,-0.133823,0.148144,0.252369,...,0.202667,0.399781,0.115776,0.107032,0.088824,-0.016651,-0.015753,-0.017067,-0.015227,-0.015529
wage_eur,0.611957,0.511929,0.778832,1.000000,0.167822,0.024463,0.065490,-0.153694,0.150848,0.236169,...,0.242122,0.435597,0.136055,0.137631,0.120798,-0.020467,-0.020429,-0.022085,-0.019032,-0.020574
age,0.458390,-0.201506,0.066446,0.167822,1.000000,0.081498,0.226143,-0.061636,0.076077,0.030409,...,0.135014,0.367661,0.153170,0.117334,0.099728,0.105135,0.110222,0.108064,0.118871,0.106004
height_cm,0.044415,0.006744,0.011040,0.024463,0.081498,1.000000,0.764081,0.056775,-0.169790,-0.422632,...,-0.326687,-0.169831,-0.053724,-0.059223,-0.069303,0.361519,0.361193,0.358273,0.360780,0.361376
weight_kg,0.140015,-0.008111,0.035725,0.065490,0.226143,0.764081,1.000000,0.009030,-0.130435,-0.357432,...,-0.244227,-0.081931,-0.039288,-0.051595,-0.063314,0.340277,0.339970,0.337763,0.341659,0.340076
league_level,-0.224736,-0.195368,-0.133823,-0.153694,-0.061636,0.056775,0.009030,1.000000,-0.021260,-0.068845,...,-0.059976,-0.126293,-0.054506,-0.046567,-0.046939,-0.016463,-0.016240,-0.019554,-0.019516,-0.016898
weak_foot,0.217422,0.167492,0.148144,0.150848,0.076077,-0.169790,-0.130435,-0.021260,1.000000,0.338108,...,0.334940,0.310992,0.044076,0.037699,0.020391,-0.228172,-0.227125,-0.223385,-0.225471,-0.228998
skill_moves,0.330951,0.285126,0.252369,0.236169,0.030409,-0.422632,-0.357432,-0.068845,0.338108,1.000000,...,0.673549,0.578220,0.143183,0.144599,0.113840,-0.609174,-0.606615,-0.604504,-0.605512,-0.608444


In [22]:
y = dataset.overall
X = dataset.drop('overall', axis=1)

In [23]:
# scaling

from sklearn.preprocessing import StandardScaler


# Step 1: Select only numeric columns
numeric_columns = X.select_dtypes(include=['float64', 'int64']).columns
X_numeric = X[numeric_columns]
X_numeric = X_numeric.fillna(X_numeric.mean())
scale = StandardScaler()
scaled = scale.fit_transform(X_numeric)


X = pd.DataFrame(scaled, columns=numeric_columns)

In [24]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_squared_log_error, r2_score

Xtrain,Xtest,Ytrain,Ytest=train_test_split(X,y,test_size=0.2,random_state=42)


In [25]:
# Multilinear Regression

from sklearn.linear_model import LinearRegression
l = LinearRegression()
l.fit(Xtrain,Ytrain)
y_pred = l.predict(Xtest)

In [26]:
Xtrain.columns

Index(['potential', 'value_eur', 'wage_eur', 'age', 'height_cm', 'weight_kg',
       'league_level', 'weak_foot', 'skill_moves', 'international_reputation',
       'pace', 'shooting', 'passing', 'dribbling', 'defending', 'physic',
       'attacking_crossing', 'attacking_finishing',
       'attacking_heading_accuracy', 'attacking_short_passing',
       'attacking_volleys', 'skill_dribbling', 'skill_curve',
       'skill_fk_accuracy', 'skill_long_passing', 'skill_ball_control',
       'movement_acceleration', 'movement_sprint_speed', 'movement_agility',
       'movement_reactions', 'movement_balance', 'power_shot_power',
       'power_jumping', 'power_stamina', 'power_strength', 'power_long_shots',
       'mentality_aggression', 'mentality_interceptions',
       'mentality_positioning', 'mentality_vision', 'mentality_penalties',
       'mentality_composure', 'defending_marking_awareness',
       'defending_standing_tackle', 'defending_sliding_tackle',
       'goalkeeping_diving', 'goalke

In [27]:
intercept = l.intercept_


coefficients = l.coef_

print(f"Intercept: {intercept}")
print(f"Coefficients: {coefficients}")

Intercept: 65.69717675765942
Coefficients: [ 3.12127591  0.31101539  0.40540059  2.32763565 -0.08440865  0.14384902
 -0.13602724 -0.01020843  0.40879687 -0.20493719 -0.69421944  0.64921001
 -0.27138573 -2.14003849  1.48526995 -0.1516236   0.45583298 -0.10124218
  0.66132002  0.62772372 -0.11103021  1.22730463  0.00707453  0.03458046
 -0.10824625  1.95928539  0.61080807  0.69791642  0.25322605  1.29593205
 -0.11626206  0.1276518   0.02141091  0.54129437  0.37535937 -0.28249931
  0.02124108 -0.29794816 -0.56627326 -0.10135064 -0.12092858  0.46908374
 -0.24088574 -0.19697558 -0.38244926  0.66456799  0.61920469  0.33144277
  0.58507503  0.76907032]


In [28]:
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")


Mean Absolute Error = 1.4291757920181882,
Mean Squared Error = 3.384335650658343,
Root Mean Squared Error = 1.839656394726565,
R2 Score = 0.9264654177621724



In [29]:
'''Polynomial Regression With Degree of 1'''
from sklearn.preprocessing import PolynomialFeatures


poly = PolynomialFeatures(degree=1)
X_poly_train = poly.fit_transform(Xtrain)
X_poly_test = poly.fit_transform(Xtest)


model = LinearRegression()
model.fit(X_poly_train,Ytrain)


y_pred = model.predict(X_poly_test)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")


Mean Absolute Error = 1.429175792018189,
Mean Squared Error = 3.3843356506583437,
Root Mean Squared Error = 1.8396563947265652,
R2 Score = 0.9264654177621726



In [ ]:
'''Polynomial Regression With Degree of 2'''
poly = PolynomialFeatures(degree=2)
X_poly_train = poly.fit_transform(Xtrain)
X_poly_test = poly.fit_transform(Xtest)


model = LinearRegression()
model.fit(X_poly_train,Ytrain)


y_pred = model.predict(X_poly_test)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")

In [ ]:
'''Regularisation Models With Ridge Model'''
from sklearn.linear_model import ElasticNet, Lasso, Ridge

# Train the model
modl = Ridge()
modl.fit(Xtrain, Ytrain)

# Test the model
y_pred = modl.predict(Xtest)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")

In [ ]:
'''Regularisation Models With Lasso Model'''
model = Lasso()
model.fit(Xtrain, Ytrain)

# Tsst the model
y_pred = model.predict(Xtest)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")

In [ ]:
'''Regularisation Models With ElasticNet Model'''
model = ElasticNet()
model.fit(Xtrain, Ytrain)

# Test the model
y_pred = model.predict(Xtest)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")

In [ ]:
'''Decision Tree Regression'''
from sklearn.tree import DecisionTreeRegressor

dtree = DecisionTreeRegressor(max_depth=12)
dtree.fit(Xtrain, Ytrain)

y_pred = dtree.predict(Xtest)
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")

In [ ]:
# Ensemble Learning
# imported several packages for different ensemble models to be created

from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, roc_auc_score, auc
import pickle as pkl
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import VotingRegressor

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor


In [ ]:
dt=DecisionTreeRegressor(criterion='absolute_error')
knn=KNeighborsRegressor(n_neighbors=7)

In [ ]:
voting=VotingRegressor(estimators=[('knn', knn), ('l', l), ('Dtree', dtree)])
voting.fit(Xtrain, Ytrain)

y_pred = voting.predict(Xtest)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")

In [ ]:
import xgboost as xgb

In [ ]:
xgb_reg = xgb.XGBRegressor()

In [ ]:
xgb_reg.fit(Xtrain, Ytrain)

y_pred = xgb_reg.predict(Xtest)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")

In [ ]:
voting=VotingRegressor(estimators=[('xbg', xgb_reg), ('Dtree', dtree)])
voting.fit(Xtrain, Ytrain)

y_pred = voting.predict(Xtest)

print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")

In [ ]:
#from sklearn.ensemble import VotingRegressor
#from xgboost import XGBRegressor
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.model_selection import GridSearchCV
#from sklearn.neighbors import KNeighborsRegressor
#from sklearn.linear_model import LinearRegression
#
# Define individual models
#xgb_reg = XGBRegressor()
#nested_voting = VotingRegressor(estimators=[('knn', KNeighborsRegressor()), ('lr', LinearRegression()), ('dt', DecisionTreeRegressor())])
#dtree = DecisionTreeRegressor()
#
# Create the main VotingRegressor
#voting = VotingRegressor(estimators=[('xgb', xgb_reg), ('nested', nested_voting), ('dt', dtree)])
#
# Parameter grid for grid search
#param_grid = {
#    'xgb__n_estimators': [100, 200],
#    'xgb__max_depth': [3, 5, 7],
#    'xgb__learning_rate': [0.01, 0.1, 0.2],
#    'nested__knn__n_neighbors': [5, 7],
#    'nested__lr__fit_intercept': [True, False],
#    'nested__dt__max_depth': [5, 10],
#    'dt__max_depth': [5, 10],
#    'dt__min_samples_split': [2, 5, 10],
#}
#
# Perform grid search
#grid_search = GridSearchCV(estimator=voting, param_grid=param_grid, scoring='r2', cv=3, n_jobs=-1)
#grid_search.fit(Xtrain, Ytrain)
#
# Predictions and evaluation
#y_pred = grid_search.predict(Xtest)
#
#print(f"""
#Best Parameters: {grid_search.best_params_}
#Mean Absolute Error = {mean_absolute_error(y_pred, Ytest)},
#Mean Squared Error = {mean_squared_error(y_pred, Ytest)},
#Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred, Ytest))},
#R2 Score = {r2_score(y_pred, Ytest)}
#""")


In [ ]:
jb.dump(voting, 'best_model.pkl', compress = 9)

In [ ]:
testor = pd.read_csv('players_22.csv', low_memory='False')

# Dropping all columns that have more than 30% null values

test_L = []
test_L_less = []
for i in testor.columns:
    if((testor[i].isnull().sum()) < (0.3*(testor.shape[0]))):
        test_L.append(i)
    else:
        test_L_less.append(i)


testor = testor[test_L]
testor = testor[dataset.columns]

#Separate the quantitative and categorical variables
test_numeric = testor.select_dtypes(include = np.number)
test_non_numeric = testor.select_dtypes(include = ['object'])

imp = IterativeImputer(max_iter = 10, random_state = 0)
test_numeric = pd.DataFrame(np.round(imp.fit_transform(test_numeric)), columns = test_numeric.columns)

encoder = ce.BinaryEncoder(cols=test_non_numeric.columns)
col_encoded = encoder.fit_transform(test_non_numeric)
test_non_numeric.dropna(inplace=True)




dataset = pd.concat([test_numeric, test_non_numeric], axis=1).reset_index(drop=True)


In [ ]:
# Testing

X1 = testor.drop(['overall'], axis=1)
y1 = testor['overall']

In [ ]:
# Step 1: Select only numeric columns
numeric_columns = X1.select_dtypes(include=['float64', 'int64']).columns
X1_numeric = X1[numeric_columns]
X1_numeric = X1_numeric.fillna(X1_numeric.mean())
scale = StandardScaler()
scaled = scale.fit_transform(X1_numeric)


X1 = pd.DataFrame(scaled, columns=numeric_columns)

In [ ]:
Xtrain,Xtest,Ytrain,Ytest=train_test_split(X1,y1,test_size=0.2,random_state=42)

In [ ]:
#Xtrain = Xtrain.iloc[:,:-5]
#Xtest = Xtest.iloc[:,:-5]
#Xtest.dropna(inplace=True)


In [ ]:
Xtrain = scale.fit_transform(Xtrain)
Xtest = scale.fit_transform(Xtest)

In [ ]:
y_pred = voting.predict(Xtest)

In [ ]:
print(f"""
Mean Absolute Error = {mean_absolute_error(y_pred,Ytest)},
Mean Squared Error = {mean_squared_error(y_pred,Ytest)},
Root Mean Squared Error = {np.sqrt(mean_squared_error(y_pred,Ytest))},
R2 Score = {r2_score(y_pred,Ytest)}
""")

In [ ]:
jb.dump(scale, 'scaler.pkl')

In [ ]:
X1.columns